In [1]:
from typing import List
from collections import Counter

import pandas as pd

from report import BenchmarkDb

from IPython.display import display
from ipywidgets import interact
import sqlalchemy
import ipywidgets as widgets

# check sqlalchemy version
if not sqlalchemy.__version__[:3] == '1.4':
    raise ValueError(f'Installed sqlalchemy version {sqlalchemy.__version__} is unsupported,'
         'this notebook requires sqlalchemy verision 1.4')


# Uncomment it if you want to turn of column hiding for large tables
# pd.set_option('display.max_columns', None)

### Requirements for interactive widgets 

In [4]:
!pip install jinja2 ipywidgets;

### EDIT ME - Basic configuration

In [14]:
# set path to your sqlite db
DB_PATH = './file.db'
DB_PATH = 'docker/e2e-hdk/results/result_database.sqlite'

# Which benchmark do you want to visualize?
BENCHMARK = ''

# for more info look https://docs.sqlalchemy.org/en/14/tutorial/engine.html
db = BenchmarkDb(engine=sqlalchemy.create_engine(f"sqlite+pysqlite:///{DB_PATH}"))

### Benchmark results overview

You can load all names of available benchmarks with this function:

In [15]:
available_benchmarks = db.load_benchmarks()

print('Currently available benchmarks in DB are:', available_benchmarks)

Currently available benchmarks in DB are: ['ny_taxi', 'census', 'plasticc']


Load all results of selected benchmark:

In [16]:
# Pick the first benchmark as an example
BENCHMARK = available_benchmarks[0]

df, ms = db.load_benchmark_results(BENCHMARK)

When you run benchmark you submit results with :
```
db.report_arbitrary(
     # any name for the benchmark, this value is stored in `df.benchmark` column
    benchmark='census',
     # any name for backend, this value is stored in `df.pandas_mode` column
    backend='unidist' or 'Modin_on_ray' or 'my_backend',
     # Unique id for several iterations, stored in `df.run_id`
    run_id=1123312,
     # Counter for each iteration, stored in `df.iteration_no`
    iteration_no=1,
     # dict with results, more details below
    name2time={'Query1': 11.2, 'Query2': 122.1, ...},
     # additional params for this run, more details below
    params={'num_cpu': 12, 'dataset_size': 1024, 'use_intel_sklearn': 'True'},
)
```
This infomation is then stored in the database and loaded in this notebook as `df` DataFrame.

Each benchmark **run** (identified with `df.run_id`) contains one or more **iterations** (`df.iteration_no`).

Column `df.benchmark` describe benchmark name, `df.date` contains date for the iteration, `df.pandas_mode` describe your backend.

If you submited some additional information with `db.report_arbitrary(..., params={'num_cpu': 12, 'dataset_size': 1024, 'other_info': 'text'})`, then it is available as columns `df[['num_cpu', 'dataset_size', 'other_info']]`

In [17]:
df[['benchmark', 'run_id', 'iteration_no', 'date', *ms[:2]]]

,benchmark,run_id,iteration_no,date,Query1,Query2
id,,,,,,
1,ny_taxi,1676371913,1,2023-02-14 10:52:25,0.111791,0.065972
2,ny_taxi,1676371913,2,2023-02-14 10:52:50,0.033158,0.073573
3,ny_taxi,1676371913,3,2023-02-14 10:53:15,0.049987,0.113234
10,ny_taxi,1676372300,1,2023-02-14 11:00:48,1.220513,2.809269
11,ny_taxi,1676372300,2,2023-02-14 11:03:09,1.211741,2.794207
12,ny_taxi,1676372300,3,2023-02-14 11:05:28,1.219335,2.780694
25,ny_taxi,1676392223,1,2023-02-14 16:30:47,0.111602,0.093559
26,ny_taxi,1676392223,2,2023-02-14 16:31:10,0.044642,0.061092
27,ny_taxi,1676392223,3,2023-02-14 16:31:33,0.047896,0.059932


`ms` variable contains names of all columns with benchmark results

In [18]:
print('ms=', ms)

df[ms]

ms= ['Query1', 'Query2', 'Query3', 'Query4', 't_readcsv']


,Query1,Query2,Query3,Query4,t_readcsv
id,,,,,
1,0.111791,0.065972,0.115464,0.454854,29.733611
2,0.033158,0.073573,0.065866,0.309440,22.088449
3,0.049987,0.113234,0.079426,0.303565,22.104618
10,1.220513,2.809269,2.019524,2.323957,137.155277
11,1.211741,2.794207,2.016075,2.322213,128.750648
12,1.219335,2.780694,2.001567,2.306541,127.001736
25,0.111602,0.093559,0.118842,0.392901,21.346447
26,0.044642,0.061092,0.055843,0.244251,20.529178
27,0.047896,0.059932,0.056370,0.257117,20.313682


In [19]:
print('There is also information about host parameter and run parameters')

df.iloc[0]

There is also information about host parameter and run parameters


benchmark                                                           ny_taxi
iteration_no                                                              1
run_id                                                           1676371913
date                                                    2023-02-14 10:52:25
server_name                                                    9d0d550a3294
architecture                                                          64bit
machine                                                              x86_64
node                                                           9d0d550a3294
os                                                                    Linux
cpu_count                                                               112
cpu_model                     Intel(R) Xeon(R) Platinum 8276L CPU @ 2.20GHz
cpu_mhz                                                            2738.734
cpu_max_mhz                                                       4000.0000
cpu_l1d_cach

In [20]:
df.iloc[:2].T

id,1,2
benchmark,ny_taxi,ny_taxi
iteration_no,1,2
run_id,1676371913,1676371913
date,2023-02-14 10:52:25,2023-02-14 10:52:50
server_name,9d0d550a3294,9d0d550a3294
architecture,64bit,64bit
machine,x86_64,x86_64
node,9d0d550a3294,9d0d550a3294
os,Linux,Linux
cpu_count,112,112


Each run might contain several iterations, if you want to aggregate results and have single measurement for each run, you will need to aggregate results like that:

In [21]:
ITERATION_AGG = 'mean'

# we assume that other params are the same across iterations of the same run, so we can take the first one
df.groupby('run_id', as_index=False).agg({c: ITERATION_AGG if c in ms else 'first' for c in df.columns}).T

,0,1,2,3
benchmark,ny_taxi,ny_taxi,ny_taxi,ny_taxi
iteration_no,1,1,1,1
run_id,1676371913,1676372300,1676392223,1676392609
date,2023-02-14 10:52:25,2023-02-14 11:00:48,2023-02-14 16:30:47,2023-02-14 16:39:20
server_name,9d0d550a3294,9d0d550a3294,0b9a845cfe4b,0b9a845cfe4b
architecture,64bit,64bit,64bit,64bit
machine,x86_64,x86_64,x86_64,x86_64
node,9d0d550a3294,9d0d550a3294,0b9a845cfe4b,0b9a845cfe4b
os,Linux,Linux,Linux,Linux
cpu_count,112,112,112,112


# Simple visualization

Simple visualization is easy to start with and good for benchmarks with several metrics without complex nesting.

In [22]:
def build_simple_table(
        df: pd.DataFrame,
        selected_metacols: List[str],
        selected_ids: List[int],
        selected_measurements: List[str]
    ):
    """Turn raw `df` with results into a more visually-appealing table with subset of results and return it. 
    
    Parameters
    ----------
    df:
        Raw dataframe with benchmark results
    selected_metacols:
        `df` columns that contain parameters of benchmark run that you want to show along with time 
        measurements. Examples: number of CPUs for the run, size of the dataset etc.
        The resulting table cols will be ordered according to the order of columns in selected_metacols,
        so it can be used to group corresponding results.
    selected_ids:
        ids, belonging to df.index, that will be selected for new table.
    selected_measurements;
        `df` columns with time measurements that require visualizaiton.
    """
    table = (
        df
        .loc[selected_ids, [*selected_metacols, *selected_measurements]]
        .sort_values(selected_metacols)
        .T
        .copy()
    )
    table.columns = [str(i) for i in table.columns]
    return table


def delta2color(v):
    if v < 0:
        c = 'green;'
    elif v > 60:
        c = 'red'
    else:
        c = '#c1c286'
    return f'color:{c};'


def display_styled(table, selected_ids, measurements):
    """Display `table` with colors and visual effects."""

    display(table
        .style
        # Hide index
        .hide(axis=1)
        # .applymap(lambda v: 'color:red;' if v > 0.25 else 'color:green;', subset=['baseline_parent_share'])
        # Coloring for deltas
        .background_gradient(axis=1, cmap="summer", subset=(measurements, table.columns))
        .applymap_index(lambda v: 'text-align:left;')
        .format('{:.4f}', subset=(measurements, table.columns))
    )

In [26]:
# Choose your benchmark
BENCHMARK = 'plasticc'
# How to aggregate results when experiment consisted of several iterations
ITERATION_AGG = 'mean'

# Columns that describe the benchmark, so that results with identical BENCHMARK_COLS can be directly compared with each other
# For instance, unidist team varied number of available cpus during the execution, so they added num_cpu, 
# that they submited with db.report_arbitrary(params={'num_cpu': 30}) during benchmark run
BENCHMARK_CONFIG_COLS = []
# Columns that describe backend, this info will be presented along with benchmark results.
# Apart from `pandas_mode` you might have something like 'library_version', 'hdk_version', 'patch_activated'
# these values have to be submited with `db.report_arbitrary(params={'library_version': 12}`
BACKEND_COLS = ['pandas_mode']

METACOLS = ['pandas_mode', 'backend_name', 'library_version']

benchmarks = db.load_benchmarks()


def present_table(benchmark, selected_measurements=None):
    # Load results
    df, measurements = db.load_benchmark_results(benchmark=benchmark)
    # Aggregate iteration results
    df = df.groupby('run_id', as_index=False).agg({c: ITERATION_AGG if c in measurements else 'first' for c in df.columns})
    
    # Select latest results for each backend
    # you can replace this with hardcoded ids
    df['latest'] = df.groupby([*BACKEND_COLS, *BENCHMARK_CONFIG_COLS])['date'].transform('rank', ascending=False)
    n_top = 1
    selected_ids = df[df['latest'] <= n_top].index

    # pick measurements
    if selected_measurements is None:    
        selected_measurements = measurements
    
    # build table for visuals
    table = build_simple_table(
        df,
        selected_metacols=BACKEND_COLS + BENCHMARK_CONFIG_COLS,
        selected_ids=selected_ids,
        selected_measurements=selected_measurements
    )
    
    # Styling
    display_styled(table, selected_ids, measurements=selected_measurements)

present_table(BENCHMARK)

pandas_mode,Modin_on_hdk,Pandas
t_connect,0.0000,0.0000
t_dmatrix,0.3993,0.3248
t_etl,14.0094,213.0443
t_infer,3.3151,3.7046
t_ml,4.6940,5.0158
t_readcsv,3.7001,212.4169
t_train_test_split,0.0336,0.1738
t_training,0.9781,0.9850


# Visualizaiton with prefix hierarchy

This is useful if you have prefix hierarchy in your benchmark like `total.preprocess.train.fe.candidate_generation.popularity`

In [12]:
def get_available_parent_prefixes(queries):
    """Learn parent prefixes for benchmarks with hierarchical measurements."""
    parent_prefixes = Counter('.'.join(tuple(n.split('.')[:-1])) for n in sorted(queries))
    parent_prefixes.pop('')

    return parent_prefixes


def filter_by_prefix(m_names, prefix: str):
    if prefix is None:
        return m_names
    prefix_splitter = tuple(prefix.split('.'))
    return [m for m in m_names if tuple(m.split('.')[:-1]) == prefix_splitter]

In [13]:
# Choose your benchmark
BENCHMARK = 'hm_fashion_recs'
# How to aggregate results when experiment consisted of several iterations
ITERATION_AGG = 'mean'

# Columns that describe the benchmark, so that results with identical BENCHMARK_COLS can be directly compared with each other
# For instance, unidist team varied number of available cpus during the execution, so they added num_cpu, 
# that they submited with db.report_arbitrary(params={'num_cpu': 30}) during benchmark run
BENCHMARK_CONFIG_COLS = []
# Columns that describe backend, this info will be presented along with benchmark results.
# Apart from `pandas_mode` you might have something like 'library_version', 'hdk_version', 'patch_activated'
# these values have to be submited with `db.report_arbitrary(params={'library_version': 12}`
BACKEND_COLS = ['pandas_mode']

METACOLS = ['pandas_mode', 'backend_name', 'library_version']

df, measurements = db.load_benchmark_results(BENCHMARK)
parent_prefixes = get_available_parent_prefixes(measurements)

if len(parent_prefixes) == 0:
    raise ValueError('No hierarchy detected in benchmark results')

@interact(
    chosen_prefix=widgets.RadioButtons(
        options=list(parent_prefixes),
        description='Prefix',
        default=1,
        disabled=False,
    )
)
def interactive_table(chosen_prefix):
    selected_measurements = [chosen_prefix, *filter_by_prefix(measurements, prefix=chosen_prefix)]
    present_table(BENCHMARK, selected_measurements=selected_measurements)

interactive(children=(RadioButtons(description='Prefix', options=('total', 'total.02-create_user_ohe_agg', 'to…